In [ ]:
%run "{os.path.dirname(os.getcwd())}\general_functions\generalFunctions.ipynb" #container

## PPTX Functions

### Slide 1

In [ ]:
def Innovation_Summary(prs, performance,grouped, numOfDuplicates, position=0):
    """
    Parameters:
    prs (Presentation): PowerPoint object.
    data : Dataframe containing performance data.
    numOfDuplicates (int): Number of slides to update.
    position (int): Starting slide index.
    """
    
    for slidenum, (group_name, row) in enumerate(grouped.iterrows()):
        product_list = row["Product"] # Extract the product list for this group
        df = performance[performance["Product"].isin(product_list)].copy() # Filter performance data for these products
        shapes = prs.slides[slidenum + position].shapes
 
        tables, charts = createTableAndChart(shapes)
        table = tables[0].table
 
        titleNumber = get_shape_number(shapes, "Summary | Innovations ")
        dataSourceNumber = get_shape_number(shapes, "SOURCE: Data Source A | Description")
        shapes[dataSourceNumber].text = data_source
        shapes[titleNumber].text = f"Summary | Innovations | {group_name}"
       
        # Remove excess rows and columns from the table
        if len(table.rows) > len(df) + 1:
            num_rows_to_remove = len(table.rows) - len(df) - 1
            table = removeRowFromTable(table, num_rows_to_remove, rowToExclude=1)
 
        # Populate the table with VSOD data
        for i, row in enumerate(table.rows):
            for j, cell in enumerate(row.cells):
                if i == 0:
                    continue
                if j == 0:
                    cell.text = str(df['Product'].iloc[i - 1])
                    cell.text_frame.paragraphs[0].runs[0].font.size = Pt(7)
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.LEFT
                elif j == 5:
                    cell.text = str(round(df['Cannibalization'].replace(np.nan, 0).iloc[i - 1] * 100, 1)) + '%'
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
                elif j == 6:
                    cell.text = str(round(df['Volume Share Index'].iloc[i - 1],1))
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
                elif j == 7:
                    cell.text = str(round(df['Unit Share Index'].iloc[i - 1],1))
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
                elif j == 8:
                    cell.text = str(round(df['Net Sales index'].replace(np.nan, 0).iloc[i - 1] , 1))
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
                elif j == 9:
                    cell.text = str(round(df['Gross Profit index'].iloc[i - 1],1))
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
                elif j == 10:
                    cell.text = str(round(df['GM%'].iloc[i - 1]* 100, 1)) + '%'
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
                elif j == 11:
                    cell.text = str(round(df['TM%'].iloc[i - 1]* 100, 1)) + '%'
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
                else :
                    continue
 
                cell.text_frame.paragraphs[0].font.size = Pt(8)
                cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
 

### Slide 2

In [ ]:
def innovationRanking(prs, dfDicSplit, performance, unitorvolshare,client, numOfDuplicates, position=0):
    """
    Updates PowerPoint slides with innovation ranking charts based on provided data.
    
    Parameters:
    prs (Presentation): PowerPoint presentation object.
    dfDicSplit (dict): Dictionary containing split DataFrames.
    performance (DataFrame): DataFrame containing performance data.
    numOfDuplicates (int): Number of duplicates to process.
    position (int, optional): Starting position of the slide to update. Default is 0.
    """
    slidenum = 0
    keyNum = 0
    # Mapping for chart indices
    chartIndex = {0: 0, 1: 2, 2: 1, 3: 3, 4: 4}
    
    # Loop through each duplicate
    for slidenum in range(numOfDuplicates):
        # Get the name of the current DataFrame
        dfName = list(duplicationList.keys())[slidenum]
        print(dfName.split('_Num')[0])
        
        # Retrieve the DataFrame from the dictionary
        df = dfDicSplit[dfName]

        
        # Access the shapes on the current slide
        shapes = prs.slides[slidenum + position].shapes
        
        # Create tables and charts on the slide
        tables, charts = createTableAndChart(shapes)
        
        # Get the title shape number and update its text
        titleNumber = get_shape_number(shapes, "Ranking SKUs when introducing each innovation one by one | Panda | Cookies'n Creme innovation")
        shapes[titleNumber - 1].text = data_source
        shapes[titleNumber].text = shapes[titleNumber].text.replace('Panda', market).replace("Cookies'n Creme innovation", dfName.split('_Num')[0])
        shapes[titleNumber].text_frame.paragraphs[0].font.size = Pt(12)
        shapes[titleNumber].text_frame.paragraphs[0].font.name = 'Nexa Bold (Headings)'
        
        # Determine the number of charts to remove based on the DataFrame
        totalChart = len(charts) - df.shape[1] + 1
        for i in range(totalChart):
            shapes._element.remove(charts[chartIndex[i]]._element)
        
        # Create tables and charts again after removal
        tables, charts = createTableAndChart(shapes)
        
        # Loop through each chart and update with DataFrame data
        for chartNum, chartElement in enumerate(charts):
            columnName = df.columns[chartNum + 1]

            # Sort DataFrame based on the current column
            dfChart = df[[unitorvolshare, columnName, 'To color']].sort_values(by=columnName, ascending=False).reset_index(drop=True)
            dfChart = dfChart[~(dfChart[columnName] == 0)]
            

            
            # Get the product name associated with the current column
            prod = performance[performance['Sheets names'] == columnName].Product.unique()[0]
            
            # Update the chart with new data
            chart = chartElement.chart
            chart_data = CategoryChartData()
            chart_data.categories = dfChart[unitorvolshare].tolist()
            chart_data.add_series(prod, dfChart[columnName])
            chart.replace_data(chart_data)
                
            # Highlight the product if it exists in the DataFrame
            for prodIndex in dfChart[dfChart["To color"] == client].index:
                point = chart.series[0].points[prodIndex]
                point.format.fill.solid()
                point.format.fill.fore_color.rgb = RGBColor(126, 202, 196)
            if len(dfChart[dfChart[unitorvolshare] == prod].index) > 0:
                prodIndex = dfChart[dfChart[unitorvolshare] == prod].index[0]
                point = chart.series[0].points[prodIndex]
                point.format.fill.solid()
                point.format.fill.fore_color.rgb = RGBColor(0, 108, 109)
                point.data_label.has_text_frame = True
                point.data_label.text_frame.text = prod

                point.data_label.text_frame.paragraphs[0].runs[0].font.color.rgb = RGBColor(0, 108, 109)
                point.data_label.text_frame.paragraphs[0].runs[0].font.size = Pt(5)



### Slide 3

In [ ]:
def innovationPotential(prs, performance, wdtext,unitorvolume, numOfDuplicates, position=0):
    """
    Updates PowerPoint slides with innovation potential charts and tables based on provided data.
    
    Parameters:
    prs (Presentation): PowerPoint presentation object.
    performance (DataFrame): DataFrame containing performance data.
    numOfDuplicates (int): Number of duplicates to process.
    position (int, optional): Starting position of the slide to update. Default is 0.
    """
    for slidenum in range(numOfDuplicates):
        # Get the unique grouping for the current slide
        group = np.sort(performance['Grouping'].astype(str).unique())[slidenum]
        
        # Filter the DataFrame for the current group
        df = performance[performance['Grouping'] == group].reset_index(drop=True)
        
        # Fill any missing values with 0
        df.fillna(0, inplace=True)
        
        # Access the shapes on the current slide
        shapes = prs.slides[slidenum + position].shapes
        
        # Create tables and charts on the slide
        tables, charts = createTableAndChart(shapes)
        
        # Get the title shape number and update its text

        titleNumber = get_shape_number(shapes, "Innovation Potential when introducing each innovation one by one | Panda | Cookies'n Creme innovation")
        shapes[titleNumber - 1].text = data_source
        shapes[titleNumber].text = shapes[titleNumber].text.replace('Panda', market).replace("Cookies'n Creme innovation", group)
        shapes[titleNumber].text_frame.paragraphs[0].font.size = Pt(12)
        shapes[titleNumber].text_frame.paragraphs[0].font.name = 'Nexa Bold (Headings)'

        
        wdbox = get_shape_number(shapes, "WD assumption = 50%")
        shapes[wdbox].text = shapes[wdbox].text.replace('Panda', market).replace("WD assumption = 50%", wdtext)
        shapes[wdbox].text_frame.paragraphs[0].font.size = Pt(8)
        shapes[wdbox].text_frame.paragraphs[0].font.name = 'Nexa Book'
        shapes[wdbox].text_frame.paragraphs[0].runs[0].font.color.rgb = RGBColor(87, 85, 85)

        
        # Update the chart with new data
        chart = charts[0].chart
        chart_data = CategoryChartData() 
        chart_data.categories = df['ProductSize']
        chart_data.add_series(df['Brand'].unique()[0] + " Share Index", df[unitorvolume+' Share Index.1'])
        chart_data.add_series(df['Brand'].unique()[0] + " Value Index", df["Revenue Index.1"])
        chart_data.add_series(df['Brand'].unique()[0] + " Gross Profit Index", df['Gross Profit index.1'])
        
        chart.replace_data(chart_data)
        
        # Update the table with new data
        table = tables[0].table
        
        # Calculate the number of columns to remove from the table
        num_columns_to_remove = (len(table.columns) - df.shape[0]) - 1
        
        # Remove the specified number of columns from the table
        table = col_cell_remove(table, num_columns_to_remove)
        
        # Loop through each row and cell in the table to update the text
        for row_number, row in enumerate(table.rows, start=0):
            for column_num, cell in enumerate(row.cells):
                if column_num == 0:
                    # Update the first column with client name
                    cell.text = cell.text.replace('Hershey’s', df['Client'].unique()[0])
                    cell.text_frame.alignment = PP_ALIGN.LEFT
                    for i in range(len(cell.text_frame.paragraphs)):
                        cell.text_frame.paragraphs[i].runs[0].font.size = Pt(8)
                        cell.text_frame.paragraphs[i].runs[0].font.bold = False
                        cell.text_frame.paragraphs[i].font.name = 'Nexa Bold'
                    continue
                elif row_number == 0:
                    # Update the second column with Cannibalization percentage
                    value = df['Cannibalization'][column_num - 1]
                    cell.text = str(int(round(value * 100, 0))) + '%'
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                elif row_number == 1:
                    # Update the third column with Revenue Index
                    value = df['Revenue Index'][column_num - 1]
                    cell.text = str(round(value, 1))
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
                else:
                    # Update the fourth column with Gross Profit Index
                    value = df['Gross Profit index'][column_num - 1]
                    cell.text = str(round(value, 1))
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
                
                # Center align the text and set font size and style
                cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                cell.text_frame.paragraphs[0].runs[0].font.size = Pt(8)
                cell.text_frame.paragraphs[0].runs[0].font.bold = False


### Slide 4

In [ ]:
def innovationSourcing(prs, performance, merged, result, sumlist, numOfDuplicates, position=0):
    """
    Updates PowerPoint slides with sourcing analysis based on provided data.

    Parameters:
    prs (Presentation): PowerPoint presentation object.
    performance (DataFrame): DataFrame containing performance data.
    sourcing (DataFrame): DataFrame containing sourcing data.
    result (dict): Dictionary mapping groups to products.
    sumlist (list): List of sums indicating the number of products per group.
    numOfDuplicates (int): Number of duplicates to process.
    position (int, optional): Starting position of the slide to update. Default is 0.
    """
    for group, slide_num in zip(result, range(len(sumlist))):
        # Access the slide based on the current position and slide number
        slide = prs.slides[slide_num + position]
        shapes = slide.shapes
        tables, charts = createTableAndChart(shapes)
        
        # Define the market and update the title and data source text
        titleNumber = get_shape_number(shapes, "Innovation | Sourcing Analysis | Panda | Cookies'n Creme innovation\nRescaled Delta Share by Product & Fair Share")
        dataSourceNumber = get_shape_number(shapes, "DATA SOURCE: Consumer Test | Dec 2023")
        shapes[dataSourceNumber].text = data_source
        # Get title shape and build updated dynamic first line
        title_shape = shapes[titleNumber]
        new_text = title_shape.text.replace('Panda', market).replace("Cookies'n Creme innovation", group)

        # Clear all paragraphs
        text_frame = title_shape.text_frame
        text_frame.clear()

        # First paragraph: dynamic heading
        p1 = text_frame.paragraphs[0]
        run1 = p1.add_run()
        run1.text = new_text.split('\n')[0]  # First line
        run1.font.name = 'Nexa Bold (Headings)'
        run1.font.size = Pt(12)

        # Second paragraph: styled metrics
        p2 = text_frame.add_paragraph()
        run2 = p2.add_run()
        run2.text = "Rescaled Delta Share by Product & "
        run2.font.name = 'Nexa Book'
        run2.font.size = Pt(10)

        run3 = p2.add_run()
        run3.text = "Fair Share"
        run3.font.name = 'Nexa Book'
        run3.font.size = Pt(10)
        run3.font.color.rgb = RGBColor(0, 161, 151)


        # Prepare the DataFrame for chart data
        base_cols = list(result[group].keys())
        new = merged[['Rescaled share diff'] + base_cols + [col + '_f' for col in base_cols]]
        new['sum'] = new[list(result[group].keys())].apply(lambda x: sum(x), axis=1)
        new_sorted = new.sort_values(by='sum', ascending=True).iloc[:11]


        # Update the charts with new data
        for i, (key, value) in zip(range(len(list(result[group].keys()))), result[group].items()):

            chart_data = CategoryChartData()
            chart_data.categories = new_sorted['Rescaled share diff']
            chart_data.add_series(value, new_sorted.iloc[:, i+1] * -1)
            charts[i].chart.replace_data(chart_data)

        # Update the table with new data
        for row_number, row in enumerate(tables[0].table.rows, start=0):
            for column_num, cell in enumerate(row.cells):
                if row_number == 0 and column_num > 0:
                    # Update the header row with product names
                    value = list(result[group].values())[column_num - 1]
                    cell.text = str(value)
                    cell.text_frame.paragraphs[0].runs[0].font.size = Pt(8)
                    cell.text_frame.paragraphs[0].runs[0].font.bold = False
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                    cell.text_frame.paragraphs[0].runs[0].font.color.rgb = RGBColor(87, 85, 85)
                if row_number > 0 and column_num == 0:
                    # Update the first column with Rescaled share diff values
                    value = new_sorted.iloc[row_number - 1, column_num]
                    cell.text = str(value)
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.LEFT
                    cell.text_frame.paragraphs[0].runs[0].font.size = Pt(7)
                    cell.text_frame.paragraphs[0].runs[0].font.bold = False
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
                if row_number > 0 and column_num > 0:
                    value = new_sorted.iloc[row_number - 1, column_num + len(base_cols)]
                    cell.text = str(round(value, 1))
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.RIGHT
                    cell.text_frame.paragraphs[0].runs[0].font.size = Pt(7)
                    cell.text_frame.paragraphs[0].runs[0].font.bold = False
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
                    cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(0,160,151)
